In [ ]:
import zipfile
import os
import pandas as pd
import numpy as np
import folium
from fastkml import kml
import geopandas as gpd
from general_fun import *
import branca

In [ ]:
engine, conn = connect_database()

In [ ]:
# read KML content 
kmz = zipfile.ZipFile(os.path.join(os.getcwd(),"gis","Werken.kmz"), 'r')  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open('doc.kml', 'r').read()  # kml content

# create KML object
k = kml.KML()
k.from_string(kml_content)

# read features from docs to folders to records and then extract geometries - in my case, Shapely points
docs = list(k.features())
folders = []
for d in docs:
    folders.extend(list(d.features()))

records = []
for f in folders:
    records.extend(list(f.features()))

geoms = [element.geometry for element in records]
names = [element.name for element in records]
descriptions = [element.description for element in records]

In [ ]:
werken = pd.read_excel(os.path.join(os.getcwd(), "gis", "werken.ods"), engine="odf", converters={'date_start':str,'date_end':str, 'project_number':str})

werken['date_start'] = pd.to_datetime(werken['date_start'], format='ISO8601', utc=False)
werken['date_end'] = pd.to_datetime(werken['date_end'], format='ISO8601', utc=False)

werken["id"] = werken.index + 1

In [ ]:
werken.head(50)

In [ ]:
projects = werken.drop_duplicates(keep='first',subset=['name'])

In [ ]:
df = gpd.read_file(os.path.join(os.getcwd(),"gis", "Werken.shp"))
df.columns = map(str.lower, df.columns)
df.rename(
    columns={
        "descript": "description",
    },
    inplace=True,
)
df.drop(columns=["description", "icon", "elevation"], inplace=True)
df["latitude"] = df.apply(
    lambda x: x["geometry"].centroid.coords.xy[1][0], axis=1
)
df["longitude"] = df.apply(
    lambda x: x["geometry"].centroid.coords.xy[0][0], axis=1
)

df["company"] = [x.rsplit(' - ', 1)[-1] for x in df["name"]]
df["name"] = [x.rsplit(' - ', 1)[0] for x in df["name"]]

company_colors = {'Alkyon Hydraulic Consultancy & Research':'green','Heijmans':'darkred','Koop Tjuchem':'lightgray','MNO Vervat':'orange','Seignette':'lightblue','Van Oord':'darkblue'}

df["company_color"] = df["company"].map(company_colors)

geoms = [[point.xy[1][0], point.xy[0][0]] for point in df.geometry]

In [ ]:
markers = df.merge(projects[['name', 'year', 'city', 'country', 'description', 'vessels', 'project_number']], how='left', on='name')

In [ ]:
sql = """select * from home.house"""
house = pd.read_sql(sql, conn)

# house = pd.read_pickle("house.pkl")
geometry = gpd.points_from_xy(house["longitude"], house["latitude"])
geo_house = gpd.GeoDataFrame(house, geometry=geometry)
geo_house_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_house.geometry]

In [ ]:
mapit = folium.Map(location=np.array(house.loc[house["location"].isin(["Boskoop"]), ["latitude", "longitude"]])[0], tiles="OpenStreetMap", zoom_start=9)

for jj, coordinates in enumerate(geo_house_list):
    mapit.add_child(folium.Marker(location=coordinates, popup=house["location"][jj], icon=folium.Icon(color='red', icon='home')))

for jj, geom in enumerate(geoms):

    html = popup_html(markers.iloc[jj, :])
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)

    mapit.add_child(folium.Marker(location=(df["geometry"][jj].y, df["geometry"][jj].x), popup=popup, icon=folium.Icon(color=df["company_color"][jj], icon='person-digging', prefix='fa')))


mapit.save(os.path.join(os.getcwd(),"html", "projects.html"))
mapit